# Chapter 1

### Exercise 1.1

**Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented.**

```scheme
>>> 10
10
>>> (+ 5 3 4)
12
>>> (- 9 1)
8
>>> (/ 6 2)
3
>>> (+ (* 2 4) (- 4 6))
6
>>> (define a 3)
>>> (define b (+ a 1))
>>> (+ a b (* a b))
19
>>> (= a b)
F
>>> (if (and (> b a) (< b (* a b)))
        b
        a)
4
>>> (cond ((= a 4) 6)
          ((= b 4) (+ 6 7 a))
          (else 25))
16
>>> (+ 2 (if (> b a) b a))
6
>>> (* (cond ((> a b) a)
             ((< a b) b)
             (else -1))
       (+ a 1))
16      
```

### Exercise 1.2

**Translate the following expression into prefix form:**
$$\frac{5+4+(2-(3-(6+\frac{4}{5})))}{3(6-2)(2-7)}$$

```scheme
(/ (+ 5 4 (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))

```

### Exercise 1.3

**Define a procedure that takes three numbers
as arguments and returns the sum of the squares of the two
larger numbers.**

In [26]:
(define (sum-squares a b)
  (+ (* a a) (* b b)))
(define (sum-squares-of-large-two a b c)
  (cond ((and (< a b) (< a c)) (sum-squares b c))
        ((and (< b a) (< b c)) (sum-squares a c))
        (else (sum-squares a b))))
(sum-squares-of-large-two 2 3 4)

25

### Exercise 1.4

**Observe that our model of evaluation allows
for combinations whose operators are compound expressions.
Use this observation to describe the behavior of the
following procedure:**
```scheme
(define (a-plus-abs-b a b)
((if (> b 0) + -) a b))
```

The procudure `a-plus-abs-b` performs the mathematical operation `a + |b|`. That is, it adds the absolute value of b to a. That is because the operator to apply to a and b is decided by the if-expression, which uses  + if b is greater than 0 and - otherwise.

### Exercise 1.5

**Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative order evaluation or normal-order evaluation. He defines the following two procedures:**
```scheme
(define (p) (p))
(define (test x y)
(if (= x 0) 0 y))
```
**Then he evaluates the expression**
```scheme
(test 0 (p))
```
**What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form if is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)**

In an applicative-order interpreter, the interpreter will get stuck trying to evaluate the expression. That is because an applicative-order evaluator will try to evaluate the arguments of `test` before substituting them into the definition. This means trying to evaluate `p`, which is infinitely recursive.

In a normal-order interpreter, Ben will observe that the expression evaluates to 0. That is because the interpreter will first substitute the arguments of `test` into its definition and then evaluate the `if` expression in the definition. As the first parameter is 0, the if expression returns 0 without ever evaluating `p`.

### Exercise 1.6

**Alyssa P. Hacker doesn’t see why `if` needs to be provided as a special form. “Why can’t I just define it as an ordinary procedure in terms of `cond`?” she asks. Alyssa’s friend Eva Lu Ator claims this can indeed be done, and she defines a new version of if:**
```scheme
(define (new-if predicate then-clause else-clause)
(cond (predicate then-clause)
(else else-clause)))
```
**Eva demonstrates the program for Alyssa:**
```scheme
(new-if (= 2 3) 0 5)
5
(new-if (= 1 1) 0 5)
0
```
**Delighted, Alyssa uses `new-if` to rewrite the square-root
program:**
```scheme
(define (sqrt-iter guess x)
(new-if (good-enough? guess x)
guess
(sqrt-iter (improve guess x) x)))
```
**What happens when Alyssa attempts to use this to compute square roots? Explain.**

When Alyssa atttempts to use this to compute square roots, the evalution gets stuck in infinite recursion. That is because `new-if` is a function instead of a special form, and that means in an applicative-order interpreter all of its arguments will be evaluated before they are substituted in the definition. In this case, the `new-if` call will need to evaluate `sqrt-iter` every time. This leads to infinite recursion with nowhere to stop.

### Exercise 1.7

**The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how `guess` changes from one iteration to the next and to stop when the change is a very small fraction of the guess. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers?**

Our good-enough? test compares the square of the guess with our radicand and returns true if the difference is lower than a certain fixed value. $0.001$ is the value used in the text. This creates problems for very small numbers because within a few iterations, the guess will be smaller than or close to $0.001$, and that guess will be returned as the square root even though, compared to the actual square root, it is not very close. For example, the square root of $0.00000004$  is $0.0002$. But if a guess is 0.0008, which is eight times the actual square root, it will be good enough for our test becuase the difference is $0.0008 - 0.0002 = 0.0006 \lt 0.001 $

Also, our test is inadequate for very large numbers because arithmetic operations are almost always performed with limited precision. Floating point numbers are stored with a limited, fixed number of bits for the significant digits and a limited, fixed number of bits for the exponent. When a number gets really large, even changing the least significant bit on by one changes the number by a large difference. As a result, limited precision floating point numbers cannot represent a lot of the intermediate values between large numbers that it can represent. So, for large enough numbers, our computations may never produce guesses that are within $1$ of the true square root, let alone $0.001$ of the square root.

In [27]:
(define (average x y)
  (/ (+ x y) 2))

(define (square x) (* x x))

(define (good-enough? oldGuess newGuess)
  (< (abs (/ (- newGuess oldGuess) oldGuess)) 0.00001))

(define (improve guess x)
  (average guess (/ x guess)))

(define (sqrt-iter guess x)
  (if (good-enough? guess (improve guess x))
      guess
      (sqrt-iter (improve guess x) x)))

(sqrt-iter 1.0 0.0064)

0.08000000019530852

In [28]:
(sqrt-iter 1.0 10000)

100.00000025490743

Yes, this square root procedure with the improved good-enough? test works much better for large and small numbers.

### Exercise 1.8

**Newton’s method for cube roots is based on the fact that if $y$ is an approximation to the cube root of $x$, then a better approximation is given by the value**

$$\frac{x/y^2 + 2y}{3}$$

**Use this formula to implement a cube-root procedure analogous to the square-root procedure.**

In [29]:
(define (improve guess x)
  (/ (+ (/ x (* guess guess)) (* 2 guess)) 3))

(define (cbrt-iter guess x)
  (if (good-enough? guess (improve guess x))
      guess
      (cbrt-iter (improve guess x) x)))
(define (cbrt x) (cbrt-iter 1.0 x))

(cbrt 27)

3.0000005410641766

### Exercise 1.9

**Each of the following two procedures defines a method for adding two positive integers in terms of the procedures `inc`, which increments its argument by 1, and `dec`, which decrements its argument by 1.**

```scheme
(define (+ a b)
(if (= a 0) b (inc (+ (dec a) b))))
(define (+ a b)
(if (= a 0) b (+ (dec a) (inc b))))
```
**Using the substitution model, illustrate the process generated by each procedure in evaluating `(+ 4 5)`. Are these processes iterative or recursive?**

The first add procedure evolves the following process :
```scheme
(+ 4 5)
(inc (+ 3 5))
(inc (inc (+ 2 5)))
(inc (inc (inc (+ 1 5))))
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
(inc (inc (inc 6)))
(inc (inc 7))
(inc 8)
9
```
This process is recursive, because the space required by the procedure grows and shrinks, and at it's highest the call stack goes as deep as the value of the first parameter.

The second add procedure evolves the following process:
```scheme
(+ 4 5)
(+ 3 6)
(+ 2 7)
(+ 1 8)
(+ 0 9)
9
```
This process is iterative, as the space required is constant throughout the life of the process.

### Exercise 1.10

**The following procedure computes a mathematical function called Ackermann’s function.**

In [1]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1) (A x (- y 1))))))

**What are the values of the following expressions?**
```scheme
(A 1 10)
(A 2 4)
(A 3 3)
```
**Consider the following procedures, where `A` is the procedure defined above:**
```scheme
(define (f n) (A 0 n))
(define (g n) (A 1 n))
(define (h n) (A 2 n))
(define (k n) (* 5 n n))
```
**Give concise mathematical definitions for the functions computed by the procedures `f`, `g`, and `h` for positive integer values of *n*. For example, `(k n)` computes $5n^2$.**

In [2]:
(A 1 10)

1024

In [3]:
(A 2 4)

65536

In [4]:
(A 3 3)

65536

For `(f n)`, the concise mathematical definition is obviously $2*n$, becasue the second `cond` clause defines `(A x y)` as `(* 2 y)` when `x` is 0.

For `(g n)`, hand substitution on a few test values shows that the mathematical operation performed is exponentiation with base 2. That is, $2^n$

For `(h n)`, similarly hand-substituting shows that it performs repeated exponentiation (tetration). So, the mathematical form is $2^{2^{2...^{2}}} (n\ times)$

Looking at the functions computed by `(A 0 y)`, `(A 1 y)`, `(A 2 y)`, etc, it appears that each function in the series repeatedly applies the previous one on itself `y` times. So they can be represented mathematically in Knuth's Up-Arrow notation:

`(A 0 y)` $ = 2*y$ 

`(A 1 y)` $ = 2\uparrow y$

`(A 2 y)` $ = 2\uparrow \uparrow y$

`(A x y)` $ = 2\uparrow^x y$

### Exercise 1.11

**A function $f$ is defined by the rule that**

$$
f(n) = 
\begin{cases}
n & \text{if} \; n \lt 3\\
f(n-1) + 2 * f(n-2) + 3 * f(n-3) & \text{if} \; n \ge 3
\end{cases}
$$

**Write a procedure that computes $f$ by means of a recursive process. Write a procedure that computes $f$ by means of an iterative process.**


Here is a procedure that computes $f$ by means of a (tree) recursive process:

In [11]:
(define (f n)
  (if (< n 3) n (+ (f (- n 1)) (* 2 (f (- n 2))) (* 3 (f (- n 3))))))

In [13]:
(f 10)

1892

Here is a procedure that computes $f$ by means of an iterative process:

In [13]:
(define (f-iter a b c nn) 
  (cond ((= nn 2) a)
        ((= nn 1) b)
        ((= nn 0) c)
        (else (f-iter (+ a (* 2 b) (* 3 c)) a b (- nn 1)))))
(define (f n) (f-iter 2 1 0 n))

In [14]:
(f 10)

1892

### Exercise 1.12

**The following pattern of numbers is called *Pascal's triangle.***

$$
\quad\quad\quad\quad1\quad\quad\quad\quad\\
\quad\quad\quad1\quad1\quad\quad\quad\\
\quad\quad1\quad2\quad1\quad\quad\\
\quad1\quad3\quad3\quad1\quad\\
1\quad4\quad6\quad4\quad1\\
\quad\quad\quad.\;.\;.\quad\quad\quad\\
$$

**The numbers at the edge of the triangle are all 1, and each number inside the triangle is the sum of the two numbers above it. Write a procedure that computes elements of Pascal's triangle by means of a recursive process.**

In [9]:
(define (pascal m n)
  (cond ((= m n) 1)
        ((= n 0) 1)
        (else (+ (pascal (- m 1) n) (pascal (- m 1) (- n 1))))))

In [12]:
(pascal 4 2)

6

### Exercise 1.13

**Prove that $Fib(n)$ is the closest integer to $\varphi^n / \sqrt{5}$, where $\varphi = (1 + \sqrt(5)) / 2$. Hint: Let $\psi = (1 - \sqrt{5}) / 2$. Use induction and the definition of the Fibonacci numbers to prove that $Fib(n) = (\varphi^n - \psi^n) / \sqrt{5}$.**

First, note that $\varphi^{n+2} = \varphi^{n+1} + \varphi^n$ and that $\varphi + \psi = 1$. These can be easily proved by using the definitions of $\varphi$ and $\psi$.

We have $Fib(0) = 0$ and $Fib(1) = 1$. So the value of $Fib(n)$ is equal to the value of Fibonacci sequence for $n = 0$ and $1$. To use mathematical induction, let us suppose that for all natural numbers up to $n+1$, $Fib(n+1)$ is equal to the $(n+1)^{th}$ fibonacci number. We will show that in that case, $Fib(n+2)$ equals the $(n+2)^{th}$ fibonacci number.

$$
\begin{align}
Fib(n) + Fib(n+1) &= \frac{\varphi^n - \psi^n}{\sqrt{5}} + \frac{\varphi^{n+1} - \psi^{n+1}}{\sqrt{5}}\\
\sqrt{5}(Fib(n) + Fib(n+1)) &= (\varphi^n - \psi^n) + (\varphi^{n+1} - \psi^{n+1})\\
&= (\varphi-\psi)(\varphi^{n-1} + \varphi^{n-2}\psi + ... + \varphi\psi^{n-2} + \psi^{n-1}) + (\varphi-\psi)(\varphi^n + \varphi^{n-1}\psi + ... + \varphi\psi^{n-1} + \psi^n)\\
&= (\varphi-\psi) ((\varphi^{n-1} + \varphi^n) + \psi (\varphi^{n-2} + \varphi^{n-1}) + ... + \psi^{n-1} (\varphi+1) + \psi^n)\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n)\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n(\varphi + \psi))\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n\varphi + \psi^{n+1}))\\
&= (\varphi^{n+2} - \psi^{n+2})\\
Fib(n) + Fib(n+1) &= \frac{\varphi^{n+2} - \psi^{n+2}}{\sqrt{5}}\\
Fib(n) + Fib(n+1) &= Fib(n+2)
\end{align}
$$

We have proved that $Fib(n+2)$ equals the $(n+2)^{th}$ fibonacci number. By the principle of mathematical induction, $Fib(n)$ equals the $n^{th}$ fibonacci number for all  $n \ge 0$.

Now, observe that $|\psi/\sqrt{5}| \lt 0.5$. So, $|\psi^n/\sqrt{5}| \lt 0.5$ for all natural $n \gt 0$. We have $Fib(n) = \varphi^n / \sqrt{5} - \psi^n / \sqrt{5}$. As $Fib(n)$ is an integer, when the second term is is negative, the non-integer parts of the two terms add up. As the non-integer part of any number cannot be greater than $1$ and that of the second term cannot be greater than $0.5$, they must add up to $1$. Because the second term is less than $0.5$, the first term must be greater. Hence, the closest integer to the first term is also the value of $Fib(n)$. When the second term is positive, their non-integer parts subtract. To make the result an integer, they have to be equal. As the second term cannot be greater than $0.5$. the non-integer part of the first term cannot be greater than $0.5$ too. Hence, the closest integer to the first term is also the value of $Fib(n)$.

### Exercise 1.14

**Draw the tree illustrating the process generated by the `count-change` procedure of Section 1.2.2 in making change for 11 cents. What are the orders of growth of space and number of steps used by this process as the amount to be changed increases?**

Here is an image depicting the tree generated by the process. Repeated executions with the same input have been merged in the tree below to keep the size manageable.

<img src="aux/tree-draw/tree.png"/>

The order of growth of the space used by this process is $\Theta(m+n)$ where $m$ is the amount and $n$ is the number of coins. This is because the maximum depth of the tree reaches $\frac{m}{d_0} + n$ where $d_0$ is the smallest denomination. 

*Note: As for the order of growth of steps used by the process, I couldn't find of a straightforward way to calculate it. I read somewhere that you need to use generating functions to solve the recurrence, and I'll update the answer after I study them and find an answer. There were a few answers on the internet but none of them felt convincing or rigorous.*

### Exercise 1.15

**The sine of an angle (specified in radians) can be computed by making use of the approximation $\sin{x} \approx x$ if $x$ is sufficiently small, and the trigonometric identity**

$$
\sin{x} = 3\sin{\frac{x}{3}} - 4\sin^3{\frac{x}{3}}
$$

**to reduce the size of the argument of sin. (For purposes of this exercise an angle is considered "sufficiently small" if its magnitude is not greater than 0.1 radians.) These ideas are incorporated in the following procedures:**

```scheme
(define (cube x) (* x x x))
(define (p x) (- (* 3 x) (* 4 (cube x))))
(define (sine angle)
  (if (not (> (abs angle) 0.1))
      angle
      (p (sine (/ angle 3.0)))))
```

**a. How many times is the procedure `p` applied when `(sine 12.15)` is evaluated?**

**b. What is the order of growth in space and number of steps (as a function of $a$) used by the processs generated by the `sine` procedure when `(sine a)` is evaluated?**

a. When `(sine 12.15)` is evaluated, the procedure `p` is applied 5 times. That is because `p` is applied on every application of `sine` when the argument has an absolute value smaller than $0.1$. Since it takes 5 divisions by 3 for $12.15$ to reach such a value ($0.5$), `p` is applied 5 times. 

b. `sine` is a linear recursive process. Both the space and number of steps required is proportional to the number of times the argument $a$ can be divided by $3$ until the absolute value is less than or equal to $0.1$. That is, $\lceil\log_3{\frac{|a|}{0.1}}\rceil$. As an order of growth in big-oh notation, we can write this as simply $\mathcal{O}(\log{|a|})$. This is the order of growth of both the space and the number of steps.

### Exercise 1.16

**Design a procedure that evolves an iterative exponentiation process that uses successive squaring and uses a logarithmic number of steps, as does fast-expt. (Hint: Using the observation that $(b^{n/2})^2 = (b^2)^{n/2}$, keep, along with the exponent $n$ and the base $b$, an additional state variable $a$, and define the state transformation in such a way that the product $ab^n$ is unchanged from state to state. At the beginning of the process $a$ is taken to be $1$, and the answer is given by the value of a at the end of the process. In general, the technique of defining an invariant quantity that remains unchanged from state to state is a powerful way to think about the design of iterative algorithms.)**

I came up with a solution that doesn't have the properties described in the hint but it's a little shorter.

In [2]:
(define (odd? n) (= (remainder n 2) 1))
(define (exp-iter b n a) (if (= n 0) a (exp-iter (* b b) (quotient n 2) (if (odd? n) (* a b) a))))
(define (exp b n) (exp-iter b n 1))

(exp 5 12)

244140625